# SQL Murder Mystery
## I use python's sqlite3 library to solve the mystery
### Author: Andy Shih
***Disclaimer: I referred to some of the Python syntax from https://www.kaggle.com/datasets/johnp47/sql-murder-mystery-database, who solved the same mystery.***

The benefits of using Python to integrate SQL queries:
- If using jupyter notebook, it is good for visualization and data analysis projects as I can run each query in each chunk and visualize the results. In addition, markdown chunks are useful too.
- I can use Pandas to visualize a data frame (a query result or just the raw table) or save a new dataframe more easily for further manipulation
- To reach the final goal -- use SQL syntax for database management and extract the table I need for further analysis. Subsequently, use Python or R for more work.

**Below is the schema of the database** 

![Schema](https://raw.githubusercontent.com/NUKnightLab/sql-mysteries/master/schema.png)

In [3]:
# import libraries
import pandas as pd
import sqlite3 as sql

In [6]:
#setting up a connection to the database
con = sql.connect('/Users/andy/software learning/SQL_projects/Muder_mystery_project/sql-murder-mystery.db')

**the crime was a "​murder"​ that occurred sometime on ​Jan.15, 2018​ 
and that it took place in ​SQL City​.**

In [7]:
query_1 = '''
SELECT * FROM crime_scene_report
WHERE date == 20180115
AND
type == 'murder'
AND
city == 'SQL City';
'''
# seting the dataframe width to max to better visualize
pd.set_option('display.max_colwidth', None)
# run the query
pd.read_sql_query(query_1, con)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


**From the report, we know that the Security footage shows that there were 2 witnesses. 
The first witness lives at the last house on "Northwestern Dr". 
The second witness, named Annabel, lives somewhere on "Franklin Ave".**

In [16]:
# the first witness
query = '''SELECT * FROM person
WHERE address_street_name == 'Northwestern Dr'
ORDER BY address_number desc;
'''
pd.read_sql_query(query, con)[:1] # There are several ways to extract the info using Pandas syntax on a data frame.
## First: just like above, index starts from 1
## Second: pd.read_sql_query(query, con).loc[:0]: loc argument, starting from 0, like general python
## Third: pd.read_sql_query(query, con).iloc[:1]: iloc argument, starting from 1

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [18]:
# the second witness
query = '''
SELECT * FROM person 
WHERE address_street_name == 'Franklin Ave'
AND
name LIKE '%Annabel%'
'''
pd.read_sql_query(query, con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


**Go to the interview table to see what they said**

In [19]:
query = '''
SELECT * FROM interview
WHERE person_id == 14887
OR person_id == 16371;'''
pd.read_sql_query(query, con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


**Go to the two gym tables to find the suspect**

In [20]:
# Merge the two clues together (the gym part)
query = '''
SELECT * FROM get_fit_now_member
JOIN get_fit_now_check_in
ON
get_fit_now_member.id == get_fit_now_check_in.membership_id
WHERE get_fit_now_member.id LIKE '48Z%'
AND
get_fit_now_member.membership_status == 'gold'
AND
get_fit_now_check_in.check_in_date == 20180109;'''
pd.read_sql_query(query, con)

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,48Z7A,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700


In [21]:
# To integrate with the car details
query = '''
SELECT * FROM person
JOIN drivers_license
ON person.license_id == drivers_license.id
WHERE plate_number LIKE '%H42W%';'''
pd.read_sql_query(query, con)

,id,name,license_id,address_number,address_street_name,ssn,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,51739,Tushar Chandra,664760,312,Phi St,137882671,664760,21,71,black,black,male,4H42WR,Nissan,Altima
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius


**Now, I am certain that the suspect is Jeremy Bowers with person id 67318**

## To test my answer (copy the below and paste it on https://mystery.knightlab.com/)

>INSERT INTO solution VALUES (1, 'Jeremy Bowers');  
        SELECT value FROM solution;

**Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, 
try querying the interview transcript of the murderer to find the real villain behind this crime. 
If you feel especially confident in your SQL skills, try to complete this final step with no more 
than 2 queries. Use this same INSERT statement with your new suspect to check your answer.**

***Here, of course I can use multiple queries, and manually extract some info, which may be clearer, but the below shows that it is possible to finish in one query.***

**Query 1**

In [27]:
query = '''
SELECT * FROM interview
WHERE person_id == 67318;'''
pd.read_sql_query(query, con)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


**Query 2** 

Below is the useful info from each table with * as the keys to use for join 
- For person table: *id, name, *license_id
- For drivers_license table: *license_id, height, hair_color, car_model
- For fb event table: *person_id, event_id, event_name, date
- To me, after trying, I think I need to use WITH to make a temp table for the event first

In [28]:
query = '''
WITH temp AS(
SELECT person_id, COUNT(*) FROM facebook_event_checkin -- Here, it could be COUNT(*) or COUNT(any col of the table)
WHERE event_name == 'SQL Symphony Concert' AND
date LIKE '201712%'
GROUP BY person_id 
HAVING COUNT(*) == 3
) -- in aggregation, only HAVING is allowed but not WHERE. ALSO, I should group by first before using conditions
--SELECT * FROM temp; -- 24556 or 99716. It worked!

SELECT person.id, person.name FROM person
JOIN drivers_license
ON person.license_id == drivers_license.id
JOIN temp
ON person.id == temp.person_id
WHERE drivers_license.height BETWEEN 65 AND 67
AND
drivers_license.hair_color == 'red'
AND
drivers_license.car_model == 'Model S';'''

pd.read_sql_query(query, con)

,id,name
0,99716,Miranda Priestly


## To test my answer (copy the below and paste it on https://mystery.knightlab.com/)

>INSERT INTO solution VALUES (1, 'Miranda Priestly');  
        SELECT value FROM solution; -- Done!

**Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!**